In [2]:
import argparse
from typing import Text
import yaml
import pandas as pd 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from imblearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


#data= pd.read_csv('data/interim/input_forest_fires.csv')

#data.head()

In [5]:
df=pd.read_csv('../data/raw/Algerian_forest_fires_dataset_UPDATE_RegionAdd.csv' ,sep=',', header='infer')

In [7]:
len(df)

244

In [12]:
data = pd.read_csv('../data/interim/input_forest_fires.csv' ,sep=',', header='infer')
data.rename(columns={' RH':'RH', ' Ws': 'Ws', 'Classes  ':'Classes', 'Rain ':'Rain'}, inplace=True)
data['Classes'] = data['Classes'].str.strip()
data_limpia = data.dropna()
columnas_categoricas = ['Classes', 'Region']
columnas_enteras = ['day', 'month', 'year']
columnas_continuas = ['Temperature', 'RH', 'Ws', 'Rain', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI']
data_transformada = data_limpia.copy()
data_transformada[columnas_categoricas] = data_limpia[columnas_categoricas].astype('category')
data_transformada[columnas_enteras] = data_limpia[columnas_enteras].astype('int64')
data_transformada[columnas_continuas] = data_limpia[columnas_continuas].astype('float64')
y = data_transformada['Classes']
data_transformada.drop(columns=['Classes'], inplace=True)
X = data_transformada.select_dtypes(include=['float64', 'int64', 'category'])
#Eliminamos las columnas que no añaden valor
X = X.drop(columns=['day', 'month', 'year'])
#Dividimos los datos
X_train_base, X_test_base, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#para las variables continuas vamos a aplicar Normalizacion usando MinMaxScaler
numericas_pipeline = Pipeline( steps=[
    ('minmax', MinMaxScaler()),
    ('scaler', StandardScaler()),
    ('PCA', PCA(n_components=0.95))
] )
#para las variables cate vamos a aplicar OneHot
catOHE_pipeline = Pipeline( steps=[
    ('OneHotEncoder', OneHotEncoder())
] )
columnas_categoricas = ['Region']
ct = columnasTransformer = ColumnTransformer(transformers=[
        ('numericas_continuas', numericas_pipeline, columnas_continuas),
        ('categoricas', catOHE_pipeline, columnas_categoricas)
        ])
X_train = ct.fit_transform(X_train_base)
X_test  = ct.transform(X_test_base)




modeloRL = LogisticRegression(penalty='l2', C=100, solver='liblinear', max_iter=1000, random_state=55)

modeloRL.fit(X_train, y_train)

modeloRL.predict(X_train)



array(['not fire', 'fire', 'fire', 'fire', 'fire', 'fire', 'fire',
       'not fire', 'fire', 'fire', 'fire', 'fire', 'not fire', 'not fire',
       'fire', 'not fire', 'not fire', 'fire', 'fire', 'fire', 'not fire',
       'fire', 'fire', 'fire', 'fire', 'not fire', 'not fire', 'not fire',
       'fire', 'fire', 'not fire', 'fire', 'fire', 'fire', 'fire', 'fire',
       'not fire', 'not fire', 'not fire', 'fire', 'not fire', 'fire',
       'not fire', 'not fire', 'not fire', 'not fire', 'not fire',
       'not fire', 'fire', 'not fire', 'not fire', 'fire', 'fire', 'fire',
       'not fire', 'fire', 'fire', 'fire', 'not fire', 'fire', 'not fire',
       'not fire', 'fire', 'fire', 'not fire', 'fire', 'fire', 'fire',
       'not fire', 'fire', 'fire', 'fire', 'not fire', 'not fire', 'fire',
       'not fire', 'fire', 'fire', 'not fire', 'not fire', 'fire',
       'not fire', 'fire', 'not fire', 'not fire', 'not fire', 'not fire',
       'not fire', 'fire', 'not fire', 'not fire', 'not f

In [13]:
print(">>Exactitud (Accuracy) de los conjuntos de Entrenamiento y Validación con Logistic Regresion:")
y_pred_trainRL = modeloRL.predict(X_train)
y_pred_testRL = modeloRL.predict(X_test)
print('accuracy-train', metrics.accuracy_score(y_train, y_pred_trainRL))
print('accuracy-test', metrics.accuracy_score(y_test, y_pred_testRL))

print("\n>>Matriz de Confusión:")
print(metrics.confusion_matrix(y_test, y_pred_testRL))

print("\n>>Reporte varias métricas:")
print(metrics.classification_report(y_test, y_pred_testRL))

>>Exactitud (Accuracy) de los conjuntos de Entrenamiento y Validación con Logistic Regresion:
accuracy-train 0.9742268041237113
accuracy-test 0.9387755102040817

>>Matriz de Confusión:
[[25  2]
 [ 1 21]]

>>Reporte varias métricas:
              precision    recall  f1-score   support

        fire       0.96      0.93      0.94        27
    not fire       0.91      0.95      0.93        22

    accuracy                           0.94        49
   macro avg       0.94      0.94      0.94        49
weighted avg       0.94      0.94      0.94        49



In [13]:
X_train.shape

(194, 8)

In [14]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7
0,-2.028354,0.457634,0.368408,-1.098172,-0.096553,0.502121,1.0,0.0
1,6.012950,2.077725,0.072559,0.778430,1.326826,0.545302,0.0,1.0
2,0.625850,-1.965689,0.249324,0.603612,-0.803313,0.521845,0.0,1.0
3,1.717305,0.329024,2.088407,-2.071165,0.826714,-0.146261,0.0,1.0
4,3.835816,1.554195,-1.191479,1.019310,0.194991,-0.231181,1.0,0.0
...,...,...,...,...,...,...,...,...
189,-3.686999,0.653949,-1.019311,-0.207262,1.378963,-0.126131,1.0,0.0
190,-3.378727,0.953954,0.282521,0.350419,0.229236,-0.277382,1.0,0.0
191,-4.114886,1.621266,1.480180,1.507668,0.820171,-0.034876,1.0,0.0
192,1.049553,-0.306956,0.175571,-0.420671,-0.138537,-0.316453,0.0,1.0


In [15]:
import pickle
modeloRL = LogisticRegression(penalty='l2', C=100, solver='liblinear', max_iter=1000, random_state=55)

modeloRL.fit(X_train,y_train)

with open('..\models\modelo.pkl', 'wb') as f:
    pickle.dump(modeloRL, f)

In [16]:
pd.read_csv('../data/processed/X_train.csv' ,sep=',', header='infer').head()

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,-2.028354,0.457634,0.368408,-1.098172,-0.096553,0.502121,1.0,0.0
1,1,6.012950,2.077725,0.072559,0.778430,1.326826,0.545302,0.0,1.0
2,2,0.625850,-1.965689,0.249324,0.603612,-0.803313,0.521845,0.0,1.0
3,3,1.717305,0.329024,2.088407,-2.071165,0.826714,-0.146261,0.0,1.0
4,4,3.835816,1.554195,-1.191479,1.019310,0.194991,-0.231181,1.0,0.0
